## Download all the data we need for Wat zei je benchmarks

Training
- news-commentary-v13.de-en
- wmt13-commoncrawl.de-en
- wmt13-europarl.de-en

Test
- wmt14-newstest2014-de-en

Test OOD
- news-commentary-v14.nl-en

In [23]:
training_sets = [
    {
        "link": "http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz",
        "subfolder": "training-parallel-nc-v13/",
        "de_file": "news-commentary-v13.de-en.de",
        "en_file": "news-commentary-v13.de-en.en"
    },
    {
        "link": "https://www.statmt.org/wmt13/training-parallel-commoncrawl.tgz",
        "subfolder": "",
        "de_file": "commoncrawl.de-en.de",
        "en_file": "commoncrawl.de-en.en"
    },
    {
        "link": "https://www.statmt.org/wmt13/training-parallel-europarl-v7.tgz",
        "subfolder": "training/",
        "de_file": "europarl-v7.de-en.de",
        "en_file": "europarl-v7.de-en.en"
    }
]
test_set = {
    "link":  "https://www.statmt.org/wmt14/test-full.tgz",
    "subfolder": "test-full/",
    "de_file": "newstest2014-deen-src.de.sgm",
    "en_file": "newstest2014-deen-src.en.sgm"
}

test_ood = {
    "link": "https://data.statmt.org/news-commentary/v14/training/news-commentary-v14.en-nl.tsv.gz",
    "subfolder": "",
    "file_name": "news-commentary-v14.en-nl.tsv",
    "file_extension": ".tsv",
    "en_file": "news-commentary-v14.en-nl.en",
    "nl_file": "news-commentary-v14.en-nl.nl"

}

import requests
import tarfile
import gzip
import shutil
import os
data_directory = "./local/data"
if not os.path.exists(data_directory):
    os.makedirs(data_directory)


In [24]:
def download_and_extract(url, de_file=None, en_file=None):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    if local_filename.endswith(".tgz") or local_filename.endswith(".tar.gz"):
        with tarfile.open(local_filename, "r:gz") as tar:
            if de_file is None or en_file is None:
                tar.extractall(path=data_directory)
            else:
                for member in tar.getmembers():
                    if member.name.endswith(de_file) or member.name.endswith(en_file):
                        tar.extract(member, path=data_directory)

    if local_filename.endswith(".gz") and not local_filename.endswith(".tar.gz"):
        with gzip.open(local_filename, 'rb') as f_in:
            with open(os.path.join(data_directory, local_filename[:-3]), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    os.remove(local_filename)

# Download and extract training sets
for dataset in training_sets:
    download_and_extract(dataset["link"], dataset["de_file"], dataset["en_file"])

# Download and extract test set
download_and_extract(test_set["link"], test_set["de_file"], test_set["en_file"])

# Download and extract ood test set
download_and_extract(test_ood["link"])

In [25]:
# convert test ood from tsv to txt
test_ood_file_path = os.path.join(data_directory, test_ood["file_name"])
if os.path.exists(test_ood_file_path):
    en_lines = []
    nl_lines = []
    with open(test_ood_file_path, 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            tab_split = line.split("\t")
            nl_lines.append(tab_split[-1])
            en_lines.append(" ".join(tab_split[:-1]) + "\n") # 32 lines have a tab in the english sentence that needs to be merged
                
    with open(os.path.join(data_directory, test_ood["en_file"]), "w") as de_f:
        de_f.writelines(en_lines)
    with open(os.path.join(data_directory, test_ood["nl_file"]), "w") as en_f:
        en_f.writelines(nl_lines)
    os.remove(test_ood_file_path)

In [26]:
from bs4 import BeautifulSoup
def sgm_to_txt(sgm_file):
    if not os.path.exists(sgm_file):
        return
    with open(sgm_file, 'r', encoding='utf-8') as sgm:
        soup = BeautifulSoup(sgm, 'html.parser')
        with open(sgm_file.replace('.sgm', ''), 'w', encoding='utf-8') as txt:
            for seg in soup.find_all('seg'):
                txt.write(seg.get_text() + '\n')
    os.remove(sgm_file)

sgm_to_txt(os.path.join(data_directory,test_set["subfolder"], test_set["de_file"]))
sgm_to_txt(os.path.join(data_directory,test_set["subfolder"], test_set["en_file"]))
test_set["de_file"] = test_set["de_file"].replace('.sgm', '')
test_set["en_file"] = test_set["en_file"].replace('.sgm', '')

In [27]:
def merge_files(datasets, subfolder,filename,second_lang="de"):
   
    target_folder = os.path.join(data_directory, subfolder)
    de_files = []
    en_files = []
    for dataset in datasets:
        de_file_path = os.path.join(data_directory, dataset["subfolder"], dataset[second_lang+"_file"])
        en_file_path = os.path.join(data_directory, dataset["subfolder"], dataset["en_file"])
        de_files.append(de_file_path)
        en_files.append(en_file_path)

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    with open(os.path.join(target_folder, filename+"."+second_lang), "w") as de_f:
        for de_file in de_files:
            with open(de_file, "r") as f:
                de_f.write(f.read())

    with open(os.path.join(target_folder, filename+".en"), "w") as en_f:
        for en_file in en_files:
            with open(en_file, "r") as f:
                en_f.write(f.read())

    for file in de_files + en_files:
        os.remove(file)

In [28]:
merge_files(training_sets, "training", "train")
merge_files([test_set], "test", "test")
merge_files([test_ood], "test_ood", "test_ood",second_lang="nl")
# Remove empty directories in data_directory
for root, dirs, files in os.walk(data_directory, topdown=False):
    for name in dirs:
        dir_path = os.path.join(root, name)
        if not os.listdir(dir_path):
            os.rmdir(dir_path)
    